# Example Go Wrapper for Seldon Core

This notebook goes through the steps to test the Go wrapper. This is presently an example of how to wrap code to run in Go in Seldon. 



Below shows example code for a REST and gRPC server in Go that handles the Seldon Core microservice API for MODELs.

In [ ]:
!pygmentize server.go

## Local Docker Test

For this test we will use the code as is which returns a hard-wired result. For real use you should copy this Go project to build your own components.


In [ ]:
!make build_docker

Run a REST test

In [ ]:
!make test_docker_rest

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 10000 -p

In [ ]:
!docker rm -f gomodel

Run a gRPC test

In [ ]:
!make test_docker_grpc

In [ ]:
!seldon-core-tester contract.json 0.0.0.0 10000 -p --grpc --tensor

In [ ]:
!docker rm -f gomodel

## Test in Minikube

In [ ]:
!minikube start --memory 4096

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core

In [ ]:
!eval $(minikube docker-env) && make build_docker

In [ ]:
!kubectl create -f resources/deployment_example.json

In [ ]:
!kubectl get seldondeployments example-go -o jsonpath='{.status}'

In [ ]:
!seldon-core-api-tester contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete